# 3-Zero-Shot-Mutation-Variant-Clinvar-Synonymous

Synonymous ClinVar variants evaluation to assesss models ability to distinguish between synonymous mutations that are pathogenic vs benign.

## Dataset Information
- **Dataset**: ClinVar Synonymous
- **Path**: `/data/validation/processed/clinvar_synom.csv`
- **Task**: Zero-shot mutation effect prediction
- **Models**: Pretrained Encodon models (80M, 600M, 1B)
- **Evaluation**: Mann-Whitney U test between healthy and disease patients (-log10p) significant test values.

In [ ]:
#!/usr/bin/env python3


import os
import sys


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import torch
from scipy.stats import mannwhitneyu
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from tqdm import tqdm


# Add project paths
sys.path.append('..')

# Import Encodon-specific modules
from src.data.metadata import MetadataFields
from src.data.mutation_dataset import MutationDataset, collate_fn
from src.data.preprocess.mutation_pred import mlm_process_item
from src.inference.encodon import EncodonInference
from src.inference.task_types import TaskTypes


print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name()}")


In [ ]:
# Configuration: Data paths and model checkpoints
DATA_INPUT_PATH = '/data/validation/processed/clinvar_synom.csv'
OUTPUT_DIR = '/data/validation/encodons_results/clinvar_synom'

# Model checkpoints for inference

ckpt_path = "/data/validation/checkpoints/release_checkpoints/release_safetensors_files"
MODEL_CHECKPOINTS = {
    "80M": f"{ckpt_path}/NV-CodonFM-Encodon-80M-v1/NV-CodonFM-Encodon-80M-v1.safetensors",
    "600m": f"{ckpt_path}/NV-CodonFM-Encodon-600M-v1/NV-CodonFM-Encodon-600M-v1.safetensors",
    "1B": f"{ckpt_path}/NV-CodonFM-Encodon-1B-v1/NV-CodonFM-Encodon-1B-v1.safetensors",
    "1B_cdwt": f"{ckpt_path}/NV-CodonFM-Encodon-Cdwt-1B-v1/NV-CodonFM-Encodon-Cdwt-1B-v1.safetensors"
}

# Inference parameters
BATCH_SIZE = 64
NUM_WORKERS = 4
CONTEXT_LENGTH = 2048

# Running inference

This part can be skipped if the predictions are already in place

### Load models checkpoints:

In [ ]:
def load_encodon_inference_model(checkpoint_path: str, device: str = "cuda") -> EncodonInference:
    """
    Load pretrained Encodon model using the inference wrapper.
    
    Args:
        checkpoint_path: Path to the pretrained checkpoint
        device: Device to load model on ('cuda' or 'cpu')
        
    Returns:
        EncodonInference object ready for mutation prediction
    """
    print(f"Loading Encodon model from: {checkpoint_path}")
    
    # Create inference wrapper
    inference_model = EncodonInference(
        model_path=checkpoint_path,
        task_type=TaskTypes.MUTATION_PREDICTION
    )
    
    # Configure the model (loads checkpoint and tokenizer)
    inference_model.configure_model()
    inference_model.eval()
    
    print(f"✅ Model loaded successfully on {device}")
    print(f"Model parameters: {sum(p.numel() for p in inference_model.model.parameters()):,}")
    print(f"Tokenizer vocabulary size: {inference_model.tokenizer.vocab_size}")
    
    return inference_model


model_loaded = False
encodon_models = {}

for size, checkpoint_path in MODEL_CHECKPOINTS.items():
    if os.path.exists(checkpoint_path):
        try:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            model = load_encodon_inference_model(checkpoint_path, device=device)
            
            # Extract model name from path
            model_name = os.path.basename(os.path.dirname(os.path.dirname(checkpoint_path)))
            display_name = f"EnCodon ({size})"
            
            encodon_models[display_name] = {
                'model': model,
                'path': checkpoint_path,
                'device': device,
                'original_name': size
            }
            print(f"✅ Successfully loaded {display_name} from: {checkpoint_path}")
            model_loaded = True
        except Exception as e:
            print(f"Failed to load from {checkpoint_path}: {e}")
            continue

if not model_loaded:
    print("❌ Could not load any Encodon model from the specified paths.")
    print("Please ensure a checkpoint exists or update the checkpoint_paths list.")
else:
    print(f"\n✅ Loaded {len(encodon_models)} models: {list(encodon_models.keys())}")


### Load ClinVar Synonymous Dataset

In [ ]:
# Load ClinVar Synonymous dataset
DATASET_CONFIG = {
    'key': 'clinvar_synom',
    'name': 'ClinVar Synonymous',
    'data_path': '/data/validation/processed/clinvar_synom.csv',
}

def load_dataset(config):
    """Load and inspect the ClinVar Synonymous dataset."""
    print(f"Loading {config['name']} dataset...")
    print(f"Path: {config['data_path']}")
    
    if not os.path.exists(config['data_path']):
        print(f"❌ Dataset not found: {config['data_path']}")
        return None
    
    try:
        # Load data using polars then convert to pandas
        data = pl.read_csv(config['data_path'], ignore_errors=True).to_pandas()
        print(f"✅ Loaded {len(data)} variants")
        print(f"Shape: {data.shape}")
        print(f"Columns: {list(data.columns)}")
        
        # Check for required columns
        required_cols = ['id', 'ref_seq', 'ref_codon', 'alt_codon', 'codon_position']
        missing_cols = [col for col in required_cols if col not in data.columns]
        
        if missing_cols:
            print(f"⚠️ Missing columns: {missing_cols}")
        else:
            print("✅ All required columns present")

        # Show sample data
        display_cols = [col for col in ['id', 'ref_codon', 'alt_codon', 'codon_position'] if col in data.columns]
        display_cols.append('label')
        
        print(f"\nSample data:")
        print(data[display_cols].head(3))

        # Set dataloader 
        
        return data
        
    except Exception as e:
        print(f"❌ Failed to load dataset: {e}")
        return None

# Load the dataset
dataset = load_dataset(DATASET_CONFIG)
print(f"\n📊 Dataset loaded: {dataset is not None}")

### Run mutation predictions

In [ ]:
def run_mutation_predictions(models, data, batch_size=32):
    """Run mutation predictions for ClinVar Alphamissense dataset."""
    if data is None or not models:
        print("❌ No data or models available")
        return {}
    
    print(f"\n=== RUNNING MUTATION PREDICTIONS FOR CLINVAR SYNONYMOUS ===")
     # Create output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    data_subset = data.copy()
    all_predictions = {}
    
    for model_name, model_info in models.items():
        original_model_name = model_info.get('original_name', model_name)
        print(f"\n--- Processing {model_name} ---")
        output_path = os.path.join(OUTPUT_DIR, f'preds_{original_model_name}.csv')
        
        # Create temporary CSV file
        temp_csv_path = f"/tmp/clinvar_synonymous_{original_model_name.replace(' ', '_')}_temp.csv"
        data_subset.to_csv(temp_csv_path, index=False)
        
        try:
            # Create MutationDataset
            mutation_dataset = MutationDataset(
                data_path=temp_csv_path,
                tokenizer=model_info['model'].tokenizer,
                process_item=mlm_process_item,
                context_length=2048,
                task="mlm",
                extract_seq=True,
                train_val_test_ratio=None
            )
            
            # Create DataLoader
            dataloader = DataLoader(
                mutation_dataset,
                batch_size=batch_size,
                shuffle=False,
                collate_fn=collate_fn,
                num_workers=0
            )
            
            # Run predictions
            all_ids = []
            all_likelihood_ratios = []
            
            model_info['model'].eval()
            model_info['model'].to(model_info['device'])
            with torch.no_grad():
                for batch in tqdm(dataloader, desc=f"{model_name} predictions"):
                    # Move batch to device
                    for key in batch:
                        if isinstance(batch[key], torch.Tensor):
                            batch[key] = batch[key].to(model_info['device'])
                    
                    # Get predictions
                    output = model_info['model'].predict_mutation(batch, ids=batch[MetadataFields.ID])
                    
                    all_ids.extend(output.ids)
                    all_likelihood_ratios.extend(output.likelihood_ratios)
            
            # Save predictions of current model
            out_df = pl.DataFrame({
                'id': np.array(all_ids),
                original_model_name: np.array(all_likelihood_ratios)
            })
            out_df = out_df.with_columns(pl.col('id').cast(pl.UInt32))
            out_df.write_csv(output_path)

            print(f"Predictions saved to {output_path}") 
            print(f"✅ Completed {len(all_ids)} predictions")
            print(f"Likelihood ratio range: [{np.min(all_likelihood_ratios):.3f}, {np.max(all_likelihood_ratios):.3f}]")
            
        except Exception as e:
            print(f"❌ Failed predictions for {model_name}: {e}")
            continue
        finally:
            # Clean up temporary file
            if os.path.exists(temp_csv_path):
                os.remove(temp_csv_path)
            
            # Offload model from GPU to free memory
            if 'model' in model_info and hasattr(model_info['model'], 'cpu'):
                model_info['model'].cpu()
                print(f"🔄 Offloaded {model_name} from GPU")
            
            # Clear GPU cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🧹 Cleared GPU cache after {model_name}")
    
    return all_predictions

# Run predictions if models and dataset are available
if 'encodon_models' in locals() and 'dataset' in locals() and dataset is not None:
    predictions = run_mutation_predictions(encodon_models, dataset)
    print(f"\n✅ Predictions completed for {len(predictions)} models")
else:
    print("❌ Cannot run predictions - missing models or dataset")
    predictions = {}

# Scoring: Pathogenic vs Benign

In [ ]:
# Load dataset with predictions from inference section
print("Loading dataset...")
dset = pl.read_csv(DATA_INPUT_PATH)
print(f"Dataset shape: {dset.shape}")
print(f"Label distribution:")
print(dset['label'].value_counts())

# Load predictions from inference paths saved in previous section
print("\nLoading predictions from inference outputs...")
for model_name in MODEL_CHECKPOINTS.keys():
    pred_path = os.path.join(OUTPUT_DIR, f'preds_{model_name}.csv')
    if os.path.exists(pred_path):
        print(f"  Loading {model_name} from {pred_path}")
        pred_df = pl.read_csv(pred_path)
        dset = dset.join(pred_df, on='id', how='left')
        print(f"    Loaded {len(pred_df)} predictions")
    else:
        print(f"  Warning: Predictions for {model_name} not found at {pred_path}")

# Add pathogenicity label
dset = dset.with_columns(
    (pl.col('label').str.to_lowercase().str.contains('pathogenic').cast(pl.Int8)).alias('is_pathogenic')
)

print(f"\nFinal dataset shape: {dset.shape}")
print(f"Columns with predictions: {[col for col in dset.columns if col in MODEL_CHECKPOINTS.keys()]}")


In [ ]:
preds_cols = [col for col in dset.columns if col in MODEL_CHECKPOINTS.keys()]
# Create separate plots for each model
for model_name in preds_cols:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    for j, (in_splice, ax) in enumerate(zip([True, False], [ax1, ax2])):
        # Filter data based on splice junction status
        filtered_data = dset.filter(pl.col('in_splice_junction') == in_splice).to_pandas()
        
        # Separate pathogenic and benign data (only from filtered data)
        pathogenic_data = filtered_data[filtered_data['is_pathogenic'] == 1][model_name].dropna()
        benign_data = filtered_data[filtered_data['is_pathogenic'] == 0][model_name].dropna()
        
        # Calculate Mann-Whitney U test
        p_value = None
        if len(pathogenic_data) > 0 and len(benign_data) > 0:
            _, p_value = mannwhitneyu(pathogenic_data, benign_data, alternative='two-sided')
        
        # Create histogram with custom labels
        sns.histplot(
            data=filtered_data,
            x=model_name,
            hue='is_pathogenic',
            hue_order=[0, 1],
            bins=50,
            kde=True,
            stat='proportion',
            common_norm=False,
            ax=ax
        )
        

        # hue_order=[0, 1] means: 0=Benign (first), 1=Pathogenic (second)
        legend_labels = [f'Benign (n={len(benign_data)})', f'Pathogenic (n={len(pathogenic_data)})']
        legend = ax.get_legend()
        if legend is not None:
            for idx, text in enumerate(legend.get_texts()):
                if idx < len(legend_labels):
                    text.set_text(legend_labels[idx])
        else:
            # If no legend exists, create one manually
            ax.legend(labels=legend_labels)
        

        splice_status = "IN" if in_splice else "NOT in"
        title = f'Histogram of {model_name} by Pathogenicity ({splice_status} Splice Junction)'
        if p_value is not None:
            title += f'\nMann-Whitney U p-value: {p_value:.2e}'
        
        ax.set_title(title)
        ax.set_xlabel(f'{model_name} likelihood ratio')
        ax.set_ylabel('Density')
    
    plt.tight_layout()
    plt.show()


In [ ]:
dset.head(2)

In [ ]:
temp = dset.filter(~pl.col('in_splice_junction')).to_pandas()
temp['is_pathogenic'] = temp['is_pathogenic'].astype(bool)

mwu_scores = []

patho = temp.loc[temp['is_pathogenic']]
non_patho = temp.loc[~temp['is_pathogenic']]
for col in preds_cols + ['phylop', 'codon_freq_ratio']:
    roc_score = roc_auc_score( temp['is_pathogenic'], temp[col])
    mwu = mannwhitneyu(patho[col], non_patho[col])
    mwu_scores.append([col, mwu.pvalue, mwu.statistic, roc_score])

mwu_scores_df = pd.DataFrame(mwu_scores, columns=['col', 'pvalue', 'statistic', 'roc_score'])
mwu_scores_df['-log10(pvalue)'] = -np.log10(mwu_scores_df['pvalue'])

plt.figure()
sns.barplot(x='col', y='-log10(pvalue)', data=mwu_scores_df)
plt.title('Pathogenic all vs Benign all\nMann-Whitney U Test -log10(pvalue)')
plt.xticks(rotation=45)
plt.figure()
plt.xticks(rotation=45)
sns.barplot(x='col', y='roc_score', data=mwu_scores_df)
plt.ylim(0.5, 1)
plt.title('Pathogenic all vs Benign all\nROC Score')

In [ ]:
temp = dset.filter(~pl.col('in_splice_junction')).to_pandas()
temp['is_pathogenic'] = temp['is_pathogenic'].astype(bool)
temp['match_key'] = temp['ref_codon'] + '>' + temp['alt_codon'] + '>' + temp['pli_bin'].astype(str) + '>' + temp['cds_offset_frac_bin'].astype(str)

mwu_scores = []

for _ in range(50):
    # Split into pathogenic and non-pathogenic
    patho = temp.loc[temp['is_pathogenic']]
    non_patho = temp.loc[~temp['is_pathogenic']]

    # Count trinuc_change in pathogenic
    patho_trinuc_counts = patho['match_key'].value_counts()


    sampled_non_patho = []
    for trinuc, count in patho_trinuc_counts.items():
        subset = non_patho[non_patho['match_key'] == trinuc]
        if len(subset) >= count:
            sampled = subset.sample(n=count)
        else:
            print(f"Not enough {trinuc} in non-pathogenic data, taking all available")
            sampled = subset  # take all available if not enough
        sampled_non_patho.append(sampled)

    sampled_non_patho = pd.concat(sampled_non_patho, ignore_index=True)
    # Combine back with pathogenic
    temp_sampled = pd.concat([patho, sampled_non_patho], ignore_index=True)

    for col in preds_cols + ['phylop', 'codon_freq_ratio']:
        roc_score = roc_auc_score( temp_sampled['is_pathogenic'], temp_sampled[col])
        mwu = mannwhitneyu(patho[col], sampled_non_patho[col])
        mwu_scores.append([col, mwu.pvalue, mwu.statistic, roc_score])


mwu_scores_df = pd.DataFrame(mwu_scores, columns=['col', 'pvalue', 'statistic', 'roc_score'])
mwu_scores_df['-log10(pvalue)'] = -np.log10(mwu_scores_df['pvalue'])

plt.figure()
sns.boxplot(x='col', y='-log10(pvalue)', data=mwu_scores_df)
plt.title('Pathogenic vs Benign matched\nMann-Whitney U Test -log10(pvalue)')
plt.xticks(rotation=45)
plt.figure()
sns.boxplot(x='col', y='roc_score', data=mwu_scores_df)
plt.title('Pathogenic vs Benign matched\nROC Score')
plt.xticks(rotation=45)